# FLIMLib Ops

## Dependencies

The FLIMJ ops live in `flimlib:flimj-ops`. This dependency has to be present in order to use the ops. You can either import the package from your maven local or the ImageJ central repository.

In [1]:
// uncomment to import from local repo
// %classpath config resolver mvnLocal
// import from ImageJ central repo
%classpath config resolver scijava.public https://maven.scijava.org/content/groups/public
// uncomment to import from ImageJ central repo
%classpath add mvn flimlib flimj-ops 2.0.2
%classpath add mvn net.imagej imagej 2.0.0-rc-72

import net.imagej.ImageJ

ij = new ImageJ()
op = ij.op()
nb = ij.notebook()

Added new repo: scijava.public


net.imagej.notebook.DefaultNotebookService [priority = 0.0]

In [2]:
// run this if dependency messed up
// %classpath reset

null

## Utility Code

Here is some utility code that helps display the multi-layer fitted images, no attention needed.

In [2]:
import net.imglib2.type.numeric.ARGBType
import net.imglib2.type.numeric.real.FloatType
import net.imagej.display.ColorTables
import net.imglib2.converter.Converters
import net.imglib2.converter.RealLUTConverter

class FancyDisplay {
    
    public channelAxis, op, nb
    
    public FancyDisplay(ij, channelAxis=3) {
        this.channelAxis = channelAxis
        this.op = ij.op()
        this.nb = ij.notebook()
    }
    
    public tableDisp(fitted, tMin=null, tMax=null, aMax=null, zMax=null) {
        def lifetimeAxis = fitted.ltAxis
        def fittedImg = fitted.paramMap
        def sampleZ = op.transform().hyperSliceView(fittedImg, lifetimeAxis, 0)
        def sampleA = []
        def sampleT = []
        for (int comp in 0..((fittedImg.dimension(lifetimeAxis) - 1) / 2 - 1)) {
            sampleA.push(op.transform().hyperSliceView(fittedImg, lifetimeAxis, comp * 2 + 1))
            sampleT.push(op.transform().hyperSliceView(fittedImg, lifetimeAxis, comp * 2 + 2))
        }

        println("Z min = " + op.stats().min(sampleZ))
        println("Z max = " + op.stats().max(sampleZ))
        for (int i in 0..sampleA.size() - 1) {
            println("A" + (i + 1) + " min = " + op.stats().min(sampleA[i]))
            println("A" + (i + 1) + " max = " + op.stats().max(sampleA[i]))
            println("Tau" + (i + 1) + " min = " + op.stats().min(sampleT[i]))
            println("Tau" + (i + 1) + " max = " + op.stats().max(sampleT[i]))
        }
        
        def pseudocolor = op.run("flim.showPseudocolor", fitted, tMin, tMax, 0, aMax);
        
        // default values from img
        zMax = zMax == null ? op.stats().max(sampleZ) : new FloatType(zMax)
        aMax = aMax == null ? op.stats().max(sampleA[0]) : new FloatType(aMax)
        tMin = tMin == null ? op.stats().min(sampleT[0]) : new FloatType(tMin)
        tMax = tMax == null ? op.stats().max(sampleT[0]) : new FloatType(tMax)
        
        def labeled = [:]
        labeled["Z"] = nb.display(sampleZ, 0, zMax.getRealFloat())
        
        for (int i in 0..sampleA.size() - 1) {
            labeled["A"   + (i + 1)] = nb.display(sampleA[i], 0, aMax.getRealFloat())
            labeled["Tau" + (i + 1)] = nb.display(sampleT[i], tMin.getRealFloat(), tMax.getRealFloat())
            labeled["Pseudocolor" + (i + 1)] = op.transform().hyperSliceView(pseudocolor, lifetimeAxis, i)
        }
        return [labeled]
    }
}

fcd = new FancyDisplay(ij)

FancyDisplay@2199df72

## Loading Dataset

Here we use the [scifio](https://imagej.net/SCIFIO) [bio-formats](https://imagej.net/Bio-Formats) plugin to load time-resolved transient data from `test2.sdt`.

In [3]:
sdtPath = "epithelial_human_FLIM.sdt"

sdt = ij.scifio().datasetIO().open(sdtPath)

[INFO] Reading SDT header


<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAEACAYAAABccqhmAAACWklEQVR42u3UMQEAAAiAMPqX1gBGcMcKcFA1wFsigAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAYAAigAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAYACAAQAGABgAYACAAQAGABgAYACAAQAGABgAYACAAQAGABgAYACAAQAGABgAYACAAQAGABgAYACAAQAGABgAYACAAQAGABgAYACAAQAGABgAYABgAIABAAYAGABgAIABAAYAGABgAIABAAYAGABgAIABAAYAGABgAIABAAYAGABgAIABAAYAGABgAIABAAYAGABgAIABAAYAGABgAIABAAYAGABgAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYAGAAgAEABgAYACAAGABgAIABAAYAGABgAIABAAYAGABgAIABAAYAGABgAIABAAYAGABgAIABAAYAGABgAIABAAYAGABgAIABAAYAGABgAIABAAYAGABgAIABAAYACAAGABgAYACAAQAGABgAYACAAQAGABgAYACAAQAGABgAYACAAQAGABgAYACAAQAGABgAYACAAQAGABgAYACAAQAGABgAYACAAQAGAFwLpBAO8jaM20wAAAAASUVORK5CYII=" />

The acquired dataset is actually a 3-dimensional image as we will be shown bellow. It appears purely dark because the notebook by default displays the first layer it sees.<br>
We now use the following snippet to "chop up" the dataset for demonstration. We also display the metadata for reference.

In [4]:
import io.scif.lifesci.SDTFormat

sdtReader = new SDTFormat.Reader()
sdtReader.setContext(ij.getContext())
sdtReader.setSource(sdtPath)
sdtMetadata = sdtReader.getMetadata()

// display the axis type of each dimension
for (d = 0; d < sdt.numDimensions(); d++) {
    printf("Dim #%d: size: %3d, type: %s\n", d, sdt.dimension(d), sdt.axis(d).type())
}

timeBase = sdtMetadata.getTimeBase()
timeBins = sdtMetadata.getTimeBins()

printf("Time base: %6f, number of bins: %d\n", timeBase, timeBins)

ij.notebook().display(op.transform().hyperSliceView(sdt, 2, 45))

[INFO] Reading SDT header
Dim #0: size: 256, type: X
Dim #1: size: 256, type: Y
Dim #2: size: 256, type: Lifetime
Time base: 10.006715, number of bins: 256


Shown above is an image of time bin 45.

## Hyperparameter Setup

Prior to fitting, we set up some fitting parameters specifying how the fitting is done. All the settings are described below. The commented settings are optional and are set to default values.

In [5]:
import flimlib.flimj.FitParams
// import flimlib.flimj.FitFunc
// import flimlib.flimj.NoiseType
// import flimlib.flimj.RestrainType


// create a new fitting parameter set
param = new FitParams()
param.transMap = sdt;
// the intensity threshold for removing noise
param.iThresh = 9.0
// // the iterative fitting routine will stop when chi-squared improvement is less than param.chisq_delta
// param.chisq_delta = 0.0001f
// // the confidence interval when calculating the error axes (95% here)
// param.chisq_percent = 95
// // the routine will also stop when chi-squared < param.chisq_target
// param.chisq_target = 1
// when does the decay start and end?
// param.fitStart = 9
// param.fitEnd = 20
// // the deacy model to use, in this case y(t) = Z + A * e^(-t / TAU)
// param.fitFunc = FitFunc.GCI_MULTIEXP_TAU
// // assume the data noise follows a Poisson distribution
// param.noise = NoiseType.NOISE_GAUSSIAN_FIT
// // the standard deviation at each data point in y
// // NB: if NoiseType.NOISE_GIVEN is used, param.sig should be passed in
// param.sig = null
// // initial Z, A_i and TAU_i (i = 1, 2, ...)
// param.param = [ 0, 0, 0, ... ]
// all three parameters above will be fitted
// param.paramFree = [ true, true, true ]
// // use the default restrain type
// param.restrain = RestrainType.ECF_RESTRAIN_DEFAULT
// the time difference between two consecutive bins (ns)
param.xInc = timeBase / timeBins
// // generates the image of return code
// param.getReturnCodeMap = false
// // generates the image of parameters
// param.getParamMap = true
// // generates the image of fitted data
// param.getFittedMap = false
// // generates the image of residuals
// param.getResidualsMap = false
// // generates the image of chi-squared
// param.getChisqMap = false
// the index of the lifetime axis (from metadata)
param.ltAxis = 2

param

xInc: 0.039089, interval: [-1, -1), intensity threshold: 9.000000, instr: null, noise: NOISE_POISSON_FIT, sig: null, param: null, paramFree: null, restrain: ECF_RESTRAIN_DEFAULT, fitFunc: flimlib.FitFuncNative@7f024c88, chisq_target: 1.000000, chisq_delta: 0.000100, chisq_percent: 95

All of the fitting ops takes the same parameter, the fitting parameter (`params`) and the Lifetime axis index (`lifetimeAxis`). The rigion of interest (`roi`) is optional (see below).

In [39]:
op.help("flim.fitLMA")

Available operations:
	(FitResults out) =
	flimlib.flimj.DefaultFitRAI$LMASingleFitRAI(
		FitParams in,
		RealMask roi?,
		RandomAccessibleInterval kernel?)

## Performing Image Fitting

Once everything is set up, the fitting routine can be easily started. The op will generate an `FitResults` object with all the per-pixel results assembled into images. Specifically, `results.paramMap` will be the image of fitted parameters if `param.getParamMap` is set to `true` (which is by default), and `results.fittedMap`, `results.residualMap`, `results.chisqMap` will be those of fitted data ($\tilde{y}$), residuals ($y-\tilde{y}$) and $\chi^2$ respectively if the corresponding `getXxMap` option is turned on.<br>

This images (`xxMap`) in `results` will be of the same size as the input dataset in X and Y directions. The result attributes (fitted parameters, $\chi^2$, etc.) for that (x, y) coordinate will be layed along the Lifetime axis. E.g. `results.paramMap(x, y, 0)` will be the *Z* (constant term) for the transient at coordinate (x, y), while `results.fittedMap(x, y, 4)` will be the fitted data of the 4th time bin ($\tilde{y}_4$) of the same pixel.

Here we demonstrate the most used ops:

### Initial Parameter Estimation (RLD)

In [ ]:
// spin!
rldRslt = op.run("flim.fitRLD", param)

// showing tau from 0 to 7
nb.display(fcd.tableDisp(rldRslt, 0, 7))

Z min = -5.5624847412109375
Z max = 0.4473683834075928
A1 min = 0.0
A1 max = 8.049470901489258
Tau1 min = 0.0
Tau1 max = 48.77220916748047
brightness_max automatically set to 144.0



Z,A1,Tau1,Pseudocolor1
,,,


### Refinement (Levenberg-Marquardt Algorithm)

#### Plaint LMA fit

In [26]:
lmaRslt = op.run("flim.fitLMA", param)

// showing tau from 0 to 7
nb.display(fcd.tableDisp(lmaRslt, 0, 7))

Z min = -53.39488983154297
Z max = 6.999826431274414
A1 min = -6.6378278732299805
A1 max = 53.53471374511719
Tau1 min = -512.1802978515625
Tau1 max = 2534.052734375
brightness_max automatically set to 144.0



Z,A1,Tau1,Pseudocolor1
,,,


The plaint LMA fit starts with an arbitrary set of initial parameters $z=0, a_i=1, \tau_i=1$. By design, the algorithm is only able to find values that locally minimizes the residuals and is therefore harder to converge compared to the following scheme:

#### LMA fit with estimated initial values

To set the initial parameters, either use `param.param` to set an array of global initial values:

In [27]:
// Z = 0, A = 1000, Tau = 0.187
param.param = [ 0, 1000, 0.18723493814468384 ]
lmaRslt = op.run("flim.fitLMA", param)
// later fits shouldn't be affected
param.param = null

// showing tau from 0 to 7
nb.display(fcd.tableDisp(lmaRslt, 0, 7))

Z min = -9.093278884887695
Z max = 40.81671905517578
A1 min = -39.968719482421875
A1 max = 9.5156888961792
Tau1 min = -2.0165974556672E13
Tau1 max = 1.00538708393984E14
brightness_max automatically set to 144.0



Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAEACAYAAABccqhmAAAkGElEQVR42u1dWa4kNw5su5/R330HH9lH8r1m0APUIC1LZAQXbckBEp5+W1VlShQZjAh++/bt23/ecv3222//+nf7NfR366rrkgtb+KMNoH3furF6X0c27OdntAsJCKPfk14PeR/t93v3cfRf6d4j9+aU4Bz1fpkgf9I9a9ea4174b6L3Jls+iLZJnz/z+++/DzceEtC0Dd7+Xi+wMItde4+WIPq2DEZbr2iGePLndGcA6B9/brDMhzTa4L3N/bk+7220qUZZgBRk2tdsA8yKB44Etd1POfYesu876qBa9feXlAC7RTsp5UYfiiVd/wS6z/XZ+N+/f//f1Uvvb0mjdzzZPAdOxj1hs7YjA4B0eqJpbdaJMXovUt0tZRZIyv8JBr8CwDMwRASCyN87aXFasxjvCW3BuHrP+dgAEAWUoKeyJdBom3y0iVFA8LmBR7iBVGI8y6GIehLdtBpYKQGYNwKCmUGEPYyOzgAsi29m5EbAOamGRwA8BCv4bPpfmYD0HmaVSN7M7XBQ6201/BoMIDK9Z09Lra02+l4PEHx+zwJotpjADmgyG4hvXPSWjOGlXA++nvecPhFIrnQifzah1hVAOwDI62pp/y6A4Js2uKfkmo3bLA8A0sKNqqfQcoL9WQkHeAJ1GglICoTaz0qZxQlB4GAEe3sS0pElgHXhRqa+ls2qIf5at8ASqKSswrsAZuEGN6f3O73GpqXGHmAKm3aj1F+NuCSBgxpvgGEOnrRR35QJIAfG5ffDF61W0oClXr+3ly6VGBL4uOPCYVLct5YCWqcpozW+yX1eL7pAEWoUk2CIPajABtn0K6nAWWrJ28oAFvsZAbyRmNfibGwNiQKtv0cIvocZiJ6APdYfoys4KRCc1grLRPnRUtKaSW72THI55JF/B1EBogw+hOKpdRlWp9CZvf3buQGzW9arD4wjxECa0k8rCaRN+STpIIQhKQNh0v/ZwSCiZr2ZJchy/j3PFiGpRZjTOO/LHvUkgwMw7EAGwNOyCu3h7QwEWjOs00C7FaCod21L61t7Jql+ADMfosbdR4E5Lf3SgJ326y3uoAmJbumNn6Ic9KLqLP+DcXFiNjSqIk0o+/bgYWtATa/GZ1I1je2I/swTDGSUiCcCa6dz5KN5GOzpzKpbLWzZALbufikc478XwevXorok9W1xgx0owJnZ2WnBayafwlrH9zJIi8rWyEZdS4u01kjsyc1Yf41ev2f4YWUjnoKUlxsyJidHZehSi1vDmlCwklx7a9pQHsEP0hoclQ/W10aMPqwtxlMosjfiAqx/giddRwNMr8xFvmdcX+vUVGy61APnNMlvBILKtHGQzXKyAOemjIBp47GdHWYNoJbxbBkLfva92zOjKNeT+I7699Y+ucdd+CbF3Qp3o6hAxWIyUSUQ+7u9n29xJQupKAQDiK7rLW+898E/brw9OS5DJrJQi9tgI5UXJ/rNn+YZgAx9iSpHI5WuG3ga7Ge2oNU4IzsvJkqi2AEDumyq9rqqFPCeqpHrVwL2ngfEjKzagdWcV59JAWGUDWRRZUd4w828+t3xgL/++ivUmMZCDWbZht503hHs9u9BIzdWA0PalJ1RAka3SSMW5A7Kw9MDW0R2gBDAIiYYJY7nO6cVpfXve2QKBshCH1oPkPR2PxJTvOIITCQdZR4cW4uBsmotDcnvYQFsJwDZ7NETdywPcdWm++OPP47tCqBrK6LbwDoLRdjpB6yJs8wUR2BbO9VnNKbLou5i2F2zHGx219bfoBpENtyMtmK6IcjOCw+lU7bkoF4QkAhESEr3FqS/DERiPQV2yubMGcDKRT86gSW/f9blVUvpkAD0Jn99i3vuytkIMxyaR3hQryXoYaVu6QiU7Rk4GsQ5EuggVF5k0faCTI+t9cbTfnOrq2knriYGinB1Snv/M5lrUQwqZhKPNMSDVWxZDBysD+804c3t1Ojsdb4sc0zqLaYtNgTg0xyANMch1HwhiwV4mo2Yp989kz+xs2vSiD04YQ/OdVHxple9ckCayovYeaHMr957GHEA3oQDWOYuzr5PswlUB2V8cQ6zGYyxkTOPVA5ohCCPKlAycTihXs1a0JIn4iYDMJYQhCJ9C1MDADJMY+fTBnHn8bTzNIahNCDUOqV3p/vOmnDcrofw0nrR2j/9MNA89Xajh47q95FZZ1tjMbZLqA2T9mCt48lvAL9OcUWKxBfY4bJSJ2k6BhCRtmWNyEaMRNnRXha5J0oGYURNu3YKLHbZMzXvN2AnS1WkX19fZiFONrnCYrXMtgCjFx/yOW6SD7Odk52o5StVlRtlgLGLefYCY3QDkquvVfCh+RBKTkQ3sgYz6tlb7pUXe5rKA0CGcWZQQNEUH/m5kSdAxGlszYxOG78Vkdqin10ruWaVPLu2CtO7AGwpMDt9RS26RsDfx0OQXXzWWj9wesvWC8li375iU6JUXYspTAaWNikoYWg3SqLJDgwMmQcdusCAgexg0GyRx6wJOB6HHCbDYjbDKSCjxx5sSQCw8rln9q+1QICWLJqV08iumdUi3NwTZ/0SbhidFnnyL70Hlt6txf0kG1CR2n8oEouUOz01Ym8uoMWLfkbGkHFJTkwRKrmscmC209XI998DEqbzAFjkeuaHYMg1FqS/xRFa0pGWCSB8A8RV+AR3IGu3JgIoznRmirb4ZgLZpI5bzk3P1l8zPWgGnUc8CJlMhO1anHLqI5OVvaKYyBT5pm7LNDnwKTJhjavv5Teg48PRjGKrGjCxOxPlkKtRaSNYiy+2O+PMMHvzynb0ZWPbVWjdqim+2JZgJsllRmfBC+hZ/fXYkmMzG66dggoegRFFnUcQYnkg0VJjtKPAnH6aT+EJpKBIRNsziy8aBNxJubjodfHZeOhwjVkfmEnbkZYhw0YbBcWR5diJTMBogo2F+mrpIFjl1y+VL//zQ/8aAsFSXr0PZBbIg6TwSOfDkuZrf/dkNyFvSYNKshki2mn1/MJgxPX5dzStiEz9tXuBeAwycwekseI7pqFeVaQXZGY7KkyJdcozSCkBUKCL3fw7pLkWwRMKEjLaBJSMdLq3YNZm04BUb3fl9QEAEUIgfe0VjCz271jaeqwikTHLQOYbnLL4GLKYd1wWe8KfhLtMep+Ys4uFCnza0EiEscf4uI1+R8MTJIPNk055pr73tHKt4pqdMq1lgR9Fw6NltKu06qi4iKlBrYAVIi3OpgdH3seoVNzCP/CWoztPM0o9TNsb2XYBWOnrSUw31pmF4fhb8RKUgrwytUQyFC+/gdGb3GSxtrQNiOq6NfnrLuQKCy6ACoeYU3tkHY5mU6i7zkyT0KjMI0IrEG3AsXuZGvwZxic/esoxoFqmbVgU0QR1hUG6BSNiUG+iEMI1YM1KVvetozJCptZHBsWeHCBCgz3C/0e08mhrbfcSgEWStSxIGmYaedJF3mPWit0S6CPuwQrs4cJugM1dx9I7z2QIRk0e1t73c0NLY8m0MWK9n0V5Ax7Dkaj2mpXxxwSujMODKU9fgR1IbSi0Rh2ls2wknhW5EScg1ChC28gSaNoOOGXGjUcJWtB7wYxHZ0pGCxhqKQFvnD0YWgIwCHfv3x/HXS8KzPZnPQvfIxrpjR/XTnR0biHjReA52VmbNGs2gnZNvM+ZzYSqO9DRAqBcdgsm4E0XM7wCIvz4elOCkQzAylvP8tTLmgbNlJMMyOuxcj/VCCT8PWl1PaqDt/oCeGmeoyGgHm8Bhg35/PcnC5JAP1RRiU5rtuAAbcnmDdLowA/UYCVqHUV6CV6dAVjNLrTMwQrqWGXHUacYC1a135fmEPQCxGi0uHTfe7gLS15CMQTpc/cMM9kBqZZhmVZPSkvpdwqZLVQNyKTzrF2WBbVmCTrWQGFR8Ukb7VdWwDo

or use `param.paramMap` to set the per-pixel initial values from a previous fit:

_Note: if both initial value settings are present, the global values will be overriden by the pixel-specific values._

### Global Analysis

In [28]:
globalRslt = op.run("flim.fitGlobal", param)

// showing tau from 0 to 7
nb.display(fcd.tableDisp(globalRslt, 0, 7))

Z min = -2.0166754722595215
Z max = 0.44340524077415466
A1 min = 0.0
A1 max = 107.58305358886719
Tau1 min = 0.0
Tau1 max = 1.0991414785385132
brightness_max automatically set to 144.0



Z,A1,Tau1,Pseudocolor1
,,"<img src=""data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAEACAYAAABccqhmAAAHZElEQVR42u3dTXbbMAxFYS7H+99gO+nJSVOnsfUL4H2DO2tjkQQuQYqS1uPx+HU1a62Rv4Xrxuescb06Nu+OzzUtMAAUFYCkBoIFANw5IZiACAAgRgKAvR2UEICAmNFHFXa08VqsfP43KgAQevDERAACDZYAAp5I5lyjGFMBxAfUnddsP4AAgL9ElC6Equ0nAEAFYN2F18p65T0BKMGsiY3vkCWYJcDNASpwoQIAQADKT4AA0HyjUF8QAKBKa9BeAoDkVQEAiLw9yNSoOMuLJRUAAAIAQAA2mC5p+7T+uKs9XZY0K2FQrCfJFSoAgLgIAEA5ASgXAQJA0LMCxpwAELQGPjPhz/jbCYJaZn2QlAoAQOBjz4tVzaAIXpbpXMASAGhTak+sXva2aev/JwBYqqgAABAAgKynFtM6RUkJia8CAMZOGu9cLwEIUgT3MQEIvhFBT4Db+ocAAHcB2BVm5MQ2qgAAewBIDh4VniUASGfc9X58/uqmc/YEAIAAABAArMX1AQEIFKBK7qgAgOAJjgAgmU7+u6VvAVcdBCU+EFIBSHazspizCQiBPqrfO7zQZglqIDcuLQEgkYLbZwkADNvLeOcZiMXukFTnXmPlbwmqAAB3AfrO0ioKVInBjmddVAAYLfiWO/MXXjMBAJYANopgPBPboQIAkagAAMlIAADGP08QI4DuM5AZFCoAkMIN9+iPnsV/4q6TgwRAFGSkAjCzgfwS26MCAJwEZG5IpsiDS4JE4BNx7iRHADsHR7JIxCPuEBBAQPCQBSwBHrXfkAJEPS6tE5WoZDzjuPCmJaxOBCwBAAQueQnAEqHkrUPLEgJAcFJMaqsKAJJR/93+O08/ImIgoJ/O65vqfakCEMgkYQ8g8DFIAawPoQKApL66byp9UJQABDKZqAAAJAqLAAQ5Lu63SmOwDKQEM8azDhy9c/BIBeAxZ6K46G9XeQkJAcDsW7Adt/ymoAVyJUgA1o/GqOgafstXhV5dkn78fwEFnCOgSi8eVQEAQyuALbv/BIBR5bilmz0ASLDYWX/zPoFBkMjaljn2NgExPtF97ckSAFChEIDggfEiAEDiEwAAAgAIAAABwFoTBAAg5UwBAUAlowIQYAIXicJbOhYSUQVgcBCf3L4LAIAAlHKoOo7PHmkVVwQAxMTfoV8pMoiQqH3btBvBgQrJdHWCqhxVABISYkcngDBzr4cAABUAlOcgAIDMRvXh01eBf/pWAAEgVigOEREAiCX6+paO0T70jpM9caUCAGwCAog8TmxgARUAYA8jsB8IAJKOACBIjEniGBIAMGxS+HzSjwAElT7Y8XfGn02RRJgiKDI8SAA6EqRU+/oPq3oEBNBLIl4KqmoBCABEDQIADtvxTxMUAeCWhLsi0SIO8uxsIwEAYUsoHwaBPYDwZY53Agp+iA0CIID7ru/s35v60lECMNvqC2NwyO+qAHB5UBOSCgBmLBAAgDsfKCIAM7y2qgAgmCUQAQCkQACCWmKQCAFAkqH4uO29FUsAkgkqAEkK45PY7mUg5lwn+YEAABCAUntpW/MHdQhA2avtQ9tcpR0qAMlljC66zorX67sAwPC4/d9vqAAgWYInmGWA0enx1QS5XblkWIeUERI3ol+M88AnPXU8uiTCd6+5vuJ3pj4/sAQY9G2v/vB1YLh3fvF+Q8lbeG9e09MKSoIgUU4qlR8EoIMy+sE4E4COkIz6NOT5gxECkCjn9Yu+zYoPFQCQXNko4Xo/zCGo9UM7ARhoSNgih6sMICEhN9YJgFzgNiCAxAmDAKBiGTgOLz8YFGG5oYc4JLj+VgEAZDJPAGaq7BnuqhI4LQ5PuwvgkVOg37sCLAEkfekA1X9D9wC6BwJpwDkA1iYEQi65d0EAaCuso18BfvfHNyqPPQHgkCBXATUdNx2k1G/19Nqf6uAz+mt7vC7JRFJxp9/EnQpAcMzoF9UAAUQmilkWBNDwtl/nt8fAHgAJDC2jSea825eVlygEoAQfnXxP730HVzjjXgmmXIVJRwVAOBCPBABIegKAz4QV2OR85YzCs3+35Xo335WSLKicXNPfSfBq4p/1HAYBBMyG/+z8+uT5/vvnQ04eEgAQcnDtmbQIQPBob3A7lkGb95IHnNOv353y6xgbHxWBQVYJuLZjr7PT5+RVAICnAUFaSHxOhAAEBIKXSgRwcIf7JBUsAQCUnrBK3gXo8N45AZXRtpT3B6gABgaz5QMIQNJDpUMAZwZC16UBWeBtARwdNIIQZKcCEDDAnUtHHQGoAKBKAQEI6C5ttPMPFUCzW3S7X+H85XDKWe+JAwEAIyosIiQAyxjAHkDNxCUOqAAAEEDiciHl/fOqJAKA5AABeFpP4oMA4MAPCAAAAUTN5t/N6mZ6EIDNuFFtIzUCODxoBJVEhAoA4ZL5uqTCBgQuoAKAcl3fEAAEOwgAkh4EAIAAABAA4FkIAgBAAAAIwI45QAAZoiAcx3sdBVYpACoAAAQAFQgIAAQFAoBEddCHAAAQAAACAEAAQNN9kskvqf0NkkpZBLvPTSQAAAAASUVORK5CYII="" />",


### Multiple component fit

In [29]:
// here we use the RLD's output as our estimation
param.paramMap = rldRslt.paramMap
lmaRslt = op.run("flim.fitLMA", param)
// later fits shouldn't be affected
param.paramMap = null

// showing tau from 0 to 7
nb.display(fcd.tableDisp(lmaRslt, 0, 7))

Z min = -53.39488983154297
Z max = 6.999826431274414
A1 min = -6.6378278732299805
A1 max = 53.53471374511719
Tau1 min = -512.1802978515625
Tau1 max = 2534.052734375
brightness_max automatically set to 144.0



Z,A1,Tau1,Pseudocolor1
,,,


For multi-exponential models ($I=\sum_{i=1}^n a_i e^{-\frac{t}{\tau_i}}$), set `param.nComp` to the number of exponential components:

In [31]:
param.nComp = 2
// fitLMA automatically performs a RLF fit if no estimation is provided
// so the following step is redundant
// param.paramMap = rldRslt.paramMap
lmaRslt = op.run("flim.fitLMA", param)
// later fits shouldn't be affected
param.nComp = 1

// showing tau from 0 to 7
nb.display(fcd.tableDisp(lmaRslt, 0, 7))

Z min = -15.20682144165039
Z max = 8.729135513305664
A1 min = -2.382659912109375
A1 max = 7.153175354003906
Tau1 min = -1.70082369536E11
Tau1 max = 6.4335894300359393E18
A2 min = -8.610066413879395
A2 max = 15.863561630249023
Tau2 min = -612.0347900390625
Tau2 max = 8.286319935488E12
brightness_max automatically set to 144.0



Z,A1,Tau1,Pseudocolor1,A2,Tau2,Pseudocolor2
,,,,,,


In [32]:
// set # of exponential components
param.nComp = 3
globalRslt = op.run("flim.fitGlobal", param)
param.nComp = 1

// showing tau from 0 to 7
nb.display(fcd.tableDisp(globalRslt, 0, 7))

Z min = -5.677427291870117
Z max = 0.30542492866516113
A1 min = 0.0
A1 max = 9.730213165283203
Tau1 min = 0.0
Tau1 max = 1.0222578048706055
A2 min = 0.0
A2 max = 9.730213165283203
Tau2 min = 0.0
Tau2 max = 0.4175545871257782
A3 min = 0.0
A3 max = 43.78595733642578
Tau3 min = 0.0
Tau3 max = 3.7073798179626465
brightness_max automatically set to 144.0



Z,A1,Tau1,Pseudocolor1,A2,Tau2,Pseudocolor2,A3,Tau3,Pseudocolor3
,,"<img src=""data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAEACAYAAABccqhmAAAIP0lEQVR42u3dS3LkOAxFUW7G+19i96TLkVFtp51KSQTwzuCOKsop4vMIUCS1Pj4+/rmCtdahf7vzObCfo/65yq93x+aO+Hz8zVXVYBIXuEEMOqg9oDoaIADTjWk8tcdgAgoWAEgc9iYAICaoJAACYoaNdq1o4/VYOfUtADBJ0P88f4qYEYDQhDFbY4sAvBN4grZ/m8AWKoB2gdcxoHY+s/UAAiAZENlTVxVkawCCBLhOAATvfHF6hE1m5IitwHpi/g1uwbQAmwNU4EIFAIAAKD/BHwQAzRcK2YIAAKqCBuMlAJC8yW8HGAMgAJQapWZ5sWQNAAABAFQFBEBQ3TL2tNuT7/quRVW7roSkM3MQV6gAAMJFACB58FQAdn+wEEDYRiACkHdWoLrPyy7cnfhcWgBsWxmv/rcTJqVV9bJCQOxYBAQce75LAKiqGRRhbRlDAloAoE2pPVHEd329igBAxaYCAEAAAERVOivNKEpKyePZVQDAWIF55XkJgCBVUQWPdXGIsU6wsdbumH1UAIC3ANQVZuTEMaoAAGsASA4eFZ4WQAARne2/8e1C1dF97n+uvdq0z14FACICAgCJCwIAScsGBECgAFVyZ5lJgNz41QJAMl38d0u/Aq7qBDM4EFIBSHazspizCMh5bDXK7h0utCEAYP9gu2gBEJ1I6V/C9hYAGLaW8coZiGW2h6S69hkrf0tQBQAEi+XqPkurKFAlBjvudVEBYPYM13Fl/s5FUEEGlZmNQAIM/Bk4DhUARiWZiYEAwGwIAiAI0+w0zdd37B9YgtrzE1UVAIiCd/QXzuI/sWvnIAEgFMRIBWBmA/FLHI8KAMTaTkCBJSEkU2KsqgAEPkELniwIwJvOkSwS8Yw3BAQgIHiIBbwF+Kh9QwoQdVyaEZWoxHjGduFDLSwjArkTCwEAglteAqBFKPnqUFtCABCcFJPGqgIQIJKR/bb/zpcfEeEIsNN1tqluSy2AQCYS1gACj0EKYDaECgCS+m7bVPqgKAEQyLH9LwiAWRORLefn9mGOzxIDYrXfbpV8sDhSgvHxrA1Hr2w8UgE45kwobvrbVS4hIQDYnlRapS9W5DfYhAAAwS0IAdA/trPZGb9X0c+//YrQb3v9Z//++f8FMDC3zflRJDgU6F0BHFn9JwAY1TJp3awBQILFzvqH1wk4QSIbW6bvLQJifKL72pMWAFChEADBA/4iAJAIxksAALgWHCAAICogABLXmEEAAKTsKSAAUMmoAASYwMVd228JgISE0pwAcA4qxU/kYrKAAXInvMV46OTHu3rqlLiKF4DOjiZ+mT449StFnAiJ2ndMbyM4UCGZOl4trgWA2Q0EACCY3gIAUAFAeQ4CINmIGcotdv79rQAVAOIE5TP4bSIiACAsyc+3GMb4jKfWWF/+wOcbz6YCEKiwCAggcjsxxwK5VRUBgEQItgMBgKQjAIJFkBCRRB+qAIBhk8LjTj8CIKjY4I2/M/3kIAFA7AYafCMADCk4+aLuc71S4qsAgGEiMvpSUB91ALwFwNDZjlATAKCkqJ3ZWxMA4EnC3ZFoERt53v0wjuAEstoxHwaBV3DhbY47AQU/GsXGZfcXMj4B2PV8V//ebnt3EPwl2QgP5vrgx8qCc2DTln0AgCQN9AEBaJoMEhSnvApkDKJmrFoACOYff1MiEQCAGBAAQS0xckVk0kdFCYAkQ2O/vfsqdglIyQQtgBkvaBxTrrPiTwIwNgCPPKcKBQQAAAGwiLSMrflBHQKg7DV2azwEQKAZx4T1nJJnSgSDMWGf3XdfjqoCgMQMFuPFweh0fDVB3O5sGdYpZYTEjbALP9fc1msNABEJ890113f8zlShWQLMsxhPL3uM/jowJOXucXy13jDhWrgvKygJgumvwVQqByoABsqwAz9n214FICDYNOj8wQgB+I3x7JMnQrAGABD7zhVAhRLOtdhaoLGLsBwNzI/nbzc9cSBBQm6sL8loPLAPAEDghEEAoGIZ6IdfHwyaGmRfna+WJBKcvQtVAIIR2HyykHKiop/uKoHTF6WXhJ03jlfGQGiz7wqwCCgJSgeocxVD1wC6Jx3RwITYUAGc5DyCUMdvna8Ptw+AQsfa4PHQ1an33m3++EZpsTR7EI4zxk9Um/qNIYhAq9Nr/1UHj5g0jv++jUCELm/3m7jTAgiOGXbpVA1UtDUBaOS8KivdhFQF4LXf4NeFVz8jcbAGELmwV7mMViGc1448a1NUAEp9z3dj8nVsn1oeBhLogOPAnEDk0GnrMSMSIeT6nwAIkqe77iTMOfsTXtnZeOR5D7+VkixIe2VYSdh+m/hXncMgAAFl9/9Wfn3y/P3350N2HroTEBgubs+qDBVAUNAk7g0wjmYVQLX+TJAQz2c27rq1+7Mi4GRlpGc7781Jt4pLBQC+dxoQAheJ50QIgIBAcKu0GPnc5/RJKmgBcHnyEwicsZFpKble20ihAspIlpT7A1QAV5602ngIhS9BAIJES+KrdAjAhj68Y+IlBTlRPEkAzjYkx4AAqADGB4wAROnWkSHMOlABgMCAAAjoTq0FIYAKoLjwPCbq21c4/7U55ap74viNAEQ7RoDN8RlfqgCIBmANoGbiEg6oAAAQgMR2IeX+eVUSAYDkAAHIPa0n8UEAMHrDD2EiAAAIgL772f0BXb8yAwIAW2/ZhQDsCzhBJRGhAkC4yPzdUuEAAtZsChUAlOtsQwAg2EEAIOlBAAQ+QAAAEADAWQgCAO0QCAAAAmCmBAhAhlAQHNt7bQVWKQAqAAAEACoQEAAQKBAASFQbfQgAoLogAAAIAAACADRtOSZfUvsvIqg7aoBIbE0AAAAASUVORK5CYII="" />",,,"<img src=""data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAEACAYAAABccqhmAAAIP0lEQVR42u3dS3LkOAxFUe7A+19t96TLkVFtp51KSQTwzuCOKsop4vMIUCS1Pj4+/rmCtdahf7vzObCfo/65yq93x+aO+Hz8zVXVYBIXuEEMOqg9oDoaIADTjWk8tcdgAgoWAEgc9iYAICaoJAACYoaNdq1o4/VYOfUtADBJ0P88f4qYEYDQhDFbY4sAvBN4grZ/m8AWKoB2gdcxoHY+s/UAAiAZENlTVxVkawCCBLhOAATvfHF6hE1m5IitwHpi/g1uwbQAmwNU4EIFAIAAKD/BHwQAzRcK2YIAAKqCBuMlAJC8yW8HGAMgAJQapWZ5sWQNAAABAFQFBEBQ3TL2tNuT7/quRVW7roSkM3MQV6gAAMJFACB58FQAdn+wEEDYRiACkHdWoLrPyy7cnfhcWgBsWxmv/rcTJqVV9bJCQOxYBAQce75LAKiqGRRhbRlDAloAoE2pPVHEd329igBAxaYCAEAAAERVOivNKEpKyePZVQDAWIF55XkJgCBVUQWPdXGIsU6wsdbumH1UAIC3ANQVZuTEMaoAAGsASA4eFZ4WQAARne2/8e1C1dF97n+uvdq0z14FACICAgCJCwIAScsGBECgAFVyZ5lJgNz41QJAMl38d0u/Aq7qBDM4EFIBSHazspizCMh5bDXK7h0utCEAYP9gu2gBEJ1I6V/C9hYAGLaW8coZiGW2h6S69hkrf0tQBQAEi+XqPkurKFAlBjvudVEBYPYM13Fl/s5FUEEGlZmNQAIM/Bk4DhUARiWZiYEAwGwIAiAI0+w0zdd37B9YgtrzE1UVAIiCd/QXzuI/sWvnIAEgFMRIBWBmA/FLHI8KAMTaTkCBJSEkU2KsqgAEPkELniwIwJvOkSwS8Yw3BAQgIHiIBbwF

## Other settings

### Region of Interest

Sometimes, instead of the whole dataset, only part of the image (e.g. the region near the nucleus) are of our interest. By specifying the `roi` parameter, we neglect unwanted parts outside of it during fitting. This greatly improves the running time on large images.

In [34]:
import net.imglib2.roi.geom.real.OpenWritableBox

min = [ 80, 80 ]
max = [ 200, 200 ]

// define our region of interest, in this case [40, 87] * [40, 87]
roi = new OpenWritableBox([ min[0] - 1, min[1] - 1 ] as double[], [ max[0] + 1, max[1] + 1 ] as double[])

net.imglib2.roi.geom.real.OpenWritableBox@43e9

We start the fitting routine the same way as before but with the `roi` parameter:

In [35]:
// fitLMA with roi
lmaRslt = op.run("flim.fitLMA", param, roi)
nb.display(fcd.tableDisp(lmaRslt, 0, 7))

Z min = -13.320141792297363
Z max = 0.34271085262298584
A1 min = -0.1491815149784088
A1 max = 13.553637504577637
Tau1 min = 0.0
Tau1 max = 511.25982666015625
brightness_max automatically set to 96.0



Z,A1,Tau1,Pseudocolor1
,,,


In the results above, all other regions outside the box is neglected.

### Binning

Binning settings are enabled by setting the binning kernel parameter. The kernel can be any image. Here we use the built-in `SQUARE_KERNEL_3`, a $3\times3$ image with each pixel valued $\frac{1}{9}$:

In [ ]:
import flimlib.flimj.FlimOps
FlimOps.SQUARE_KERNEL_3

// spin!
lmaRslt = ij.op().run("flim.fitLMA", param, null, FlimOps.SQUARE_KERNEL_3)

nb.display(fcd.tableDisp(lmaRslt, 0, 7))

Z min = -14.978351593017578
Z max = 2.4580700397491455
A1 min = -0.0870462954044342
A1 max = 41.30400085449219
Tau1 min = -24.23126983642578
Tau1 max = 749.5975341796875
brightness_max automatically set to 1161.0



Z,A1,Tau1,Pseudocolor1
,,,
